#Search with NLP project

#install packages

In [ ]:
!pip install PyPDF2
!pip install sentence_transformers

#download data

In [ ]:
 import os
# os.mkdir('/content/pdfs')

In [ ]:
!wget -O pdfs/ml.pdf  https://link.springer.com/content/pdf/10.1007/s42979-021-00592-x.pdf
!wget -O pdfs/Tree.pdf https://core.ac.uk/download/pdf/250137461.pdf


--2024-09-05 11:00:28--  https://link.springer.com/content/pdf/10.1007/s42979-021-00592-x.pdf
Resolving link.springer.com (link.springer.com)... 151.101.0.95, 151.101.64.95, 151.101.128.95, ...
Connecting to link.springer.com (link.springer.com)|151.101.0.95|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://idp.springer.com/authorize?response_type=cookie&client_id=springerlink&redirect_uri=https%3A%2F%2Flink.springer.com%2Fcontent%2Fpdf%2F10.1007%2Fs42979-021-00592-x.pdf [following]
--2024-09-05 11:00:29--  https://idp.springer.com/authorize?response_type=cookie&client_id=springerlink&redirect_uri=https%3A%2F%2Flink.springer.com%2Fcontent%2Fpdf%2F10.1007%2Fs42979-021-00592-x.pdf
Resolving idp.springer.com (idp.springer.com)... 151.101.0.95, 151.101.64.95, 151.101.128.95, ...
Connecting to idp.springer.com (idp.springer.com)|151.101.0.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://link.springer.com/conte

#import packages

In [ ]:
from nltk.tokenize import sent_tokenize
from PyPDF2 import PdfReader
# import faiss
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.cluster.util import cosine_distance
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sentence_transformers import SentenceTransformer
import os
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import os
STOP = stopwords.words("english")
text = ""

for filename in os.listdir('/content/pdfs'):
  if '.pdf' in filename:
    reader = PdfReader(f'/content/pdfs/{filename}')
  for page in reader.pages:
    text += page.extract_text() + "\n"


#tokenize

In [ ]:
sentences = sent_tokenize(text)

In [ ]:
len(sentences)

2861

#clean stopwords

In [ ]:
def sent_cleaner(sent):
  words = word_tokenize(sent)
  cleaned_words = [word for word in words if word.lower() not in stopwords.words('english') and word.isalpha()]
  # for word in words:
  #   if word.isalpha() and not(word.lower() in stopwords):
  #     output.append(word)
  return ' '.join(cleaned_words)

In [ ]:
sent_cleaner(sentences[10])

'instance current tronic world wealth various kinds data Internet Things IoT data cybersecurity data smart city data business data smartphone data social media data health data data many'

In [ ]:
cleaned_sents = list(map(sent_cleaner,sentences))

In [ ]:
cleaned_sents[10]

'instance current tronic world wealth various kinds data Internet Things IoT data cybersecurity data smart city data business data smartphone data social media data health data data many'

In [ ]:
def stemize_lemma(sent):
  porter = PorterStemmer()
  lema = WordNetLemmatizer()
  words = word_tokenize(sent)
  output = []
  for word in words:
      output.append(lema.lemmatize(porter.stem(word)))
  return ' '.join(output)

In [ ]:
stemize_lemma(cleaned_sents[10])

'instanc current tronic world wealth variou kind data internet thing iot data cybersecur data smart citi data busi data smartphon data social medium data health data data mani'

In [ ]:
cleaned_sents = list(map(stemize_lemma,cleaned_sents))

In [ ]:
# vectorizer = TfidfVectorizer(norm = None)
# X = vectorizer.fit_transform(cleaned_sents).torray()
# X.shape
# from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
vectorizer = TfidfVectorizer(norm=None,max_features=700)

# Fit and transform the cleaned sentences
X = vectorizer.fit_transform(cleaned_sents).toarray()

print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
def query_preprocess(query):
  query = sent_cleaner(query)
  query = stemize_lemma(query)
  # Vectorizer = TfidfVectorizer(norm = None,max_features=700)
  features = vectorizer.transform([query]).toarray()
  # features = Vectorizer.transform
  return features

In [ ]:
query = 'Decision trees are one of the most powerful'
features = query_preprocess(query)

In [ ]:
print(features)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.   

In [ ]:
features.shape

(1, 700)

In [ ]:
features[0].shape

(700,)

In [ ]:
import pandas as pd

In [ ]:
def search(query):
  # feautres = query_preprocess(query)
  # query = 'Decision trees are one of the most powerful'
  features = query_preprocess(query)
  rate_list = []

  for i in range(len(X)):
    rate = cosine_similarity(features[0].reshape(1,-1),X[i].reshape(1,-1))
    rate_list.append(rate[0][0])

  df = pd.DataFrame(zip(rate_list,sentences),columns = ['rate', 'sent'])
  df2 = df.sort_values(by='rate',ascending=False).head(3)['sent']
  output = ''
  for txt in df2.values:
    output+= txt
    output += '\n\n'
  return output,df

,rate,sent


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

In [ ]:
print(search('what is Decision trees')[1])

      rate                                               sent
0      0.0                                               Vol.
1      0.0  :(0123456789)SN Computer Science (2021) 2:160 ...
2      0.0  To intel-\nligently analyze these data and dev...
3      0.0  Various types of machine learning algorithms s...
4      0.0  Besides, the deep learning, which \nis part of...
...    ...                                                ...
2856   0.0                    Morgan Kaufmann, San Francisco.
2857   0.0                                         Retrieved.
2858   0.0  Retrieved from \nhttp://books.google.com/books...
2859   0.0                             Breast Cancer Dataset.
2860   0.0  Retrieved December 12, 2014, from \nhttps://ar...

[2861 rows x 2 columns]


In [ ]:
print(search('what is Reinforcement Learning')[0])

Reinforcement Learning  ................................ ................................ ...................  10 
2.3.

Reinforcement learning: 
a survey.

Reinforcement Learning
Reinforcement learning (RL) is a machine learning tech-
nique that allows an agent to learn by trial and error in an 
interactive environment using input from its actions and 
experiences.


